In [1]:
from pathlib import Path
import cv2
from PIL import Image
import pytesseract
from corpus_unpdf import get_img

In [2]:
ROI = "temp/roi.png"
HEADER_TOP = 350
WIDTH_HEIGHT = (50,10)
COLOR_GREEN = (36, 255, 12)
LINE_INDICATOR_HEIGHT = 20
LINE_INDICATOR_WIDTH = 300

Executing <Task pending name='Task-3' coro=<Kernel.dispatch_queue() running at /Users/mv/Code/nb/.venv/lib/python3.11/site-packages/ipykernel/kernelbase.py:512> wait_for=<Future pending cb=[Task.task_wakeup()] created at /Users/mv/Code/nb/.venv/lib/python3.11/site-packages/tornado/queues.py:248> cb=[IOLoop.add_future.<locals>.<lambda>() at /Users/mv/Code/nb/.venv/lib/python3.11/site-packages/tornado/ioloop.py:687] created at /Users/mv/.pyenv/versions/3.11.1/lib/python3.11/asyncio/tasks.py:670> took 0.226 seconds


In [3]:
PAGE_NUM = 2
im = get_img(Path().cwd() / "data" / "data1.pdf", pagenum=PAGE_NUM)

In [4]:
import numpy
opencvImage = cv2.cvtColor(numpy.array(im), cv2.COLOR_RGB2BGR)
base_image = opencvImage.copy()
im_h, im_w, im_d = opencvImage.shape

Executing <Task pending name='Task-3' coro=<Kernel.dispatch_queue() running at /Users/mv/Code/nb/.venv/lib/python3.11/site-packages/ipykernel/kernelbase.py:512> wait_for=<Future pending cb=[Task.task_wakeup()] created at /Users/mv/Code/nb/.venv/lib/python3.11/site-packages/tornado/queues.py:248> cb=[IOLoop.add_future.<locals>.<lambda>() at /Users/mv/Code/nb/.venv/lib/python3.11/site-packages/tornado/ioloop.py:687] created at /Users/mv/.pyenv/versions/3.11.1/lib/python3.11/asyncio/tasks.py:670> took 1.094 seconds


In [5]:
gray = cv2.cvtColor(opencvImage, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (7,7), 0)
thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

In [6]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, WIDTH_HEIGHT)
dilate = cv2.dilate(thresh, kernel, iterations=1)
cv2.imwrite("temp/sample_dilated.png", dilate)

True

In [7]:
cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
cnts = sorted(cnts, key=lambda x: cv2.boundingRect(x)[1])

In [8]:
for c in cnts:
    x, y, w, h = cv2.boundingRect(c)
    if w > LINE_INDICATOR_WIDTH and h < LINE_INDICATOR_HEIGHT: # spot footnote indicator, see related kernel (50, 10)
        print(f"{x=}, {y=}, {w=}, {h=}")
        cv2.rectangle(opencvImage, (x,y), (x+w, y+h), COLOR_GREEN, 2)
        roi = base_image[HEADER_TOP : y + h, 0 : x + im_w]
cv2.imwrite("temp/sample_boxes.png", opencvImage)

x=426, y=3485, w=499, h=13


True

In [14]:
from PIL import Image
try: # roi found
    cv2.imwrite(ROI, roi)
    ocr_result = pytesseract.image_to_string(roi)
    pilimg = Image.open(ROI)
    pilimg.save(
        "temp/temp.pdf", "PDF" ,resolution=100.0, save_all=True
    )	
    print("Reveal ROI")    

except NameError: # no roi, just remove header
    cv2.imwrite(ROI, base_image)
    ocr_result = pytesseract.image_to_string(base_image[HEADER_TOP:im_h, 0:im_w])
    pilimg = Image.open(ROI)
    pilimg.save(
        "temp/temp.pdf", "PDF" ,resolution=300.0, save_all=True
    )	
    print("Reveal base")


Reveal ROI


Executing <Task pending name='Task-3' coro=<Kernel.dispatch_queue() running at /Users/mv/Code/nb/.venv/lib/python3.11/site-packages/ipykernel/kernelbase.py:512> wait_for=<Future pending cb=[Task.task_wakeup()] created at /Users/mv/Code/nb/.venv/lib/python3.11/site-packages/tornado/queues.py:248> cb=[IOLoop.add_future.<locals>.<lambda>() at /Users/mv/Code/nb/.venv/lib/python3.11/site-packages/tornado/ioloop.py:687] created at /Users/mv/.pyenv/versions/3.11.1/lib/python3.11/asyncio/tasks.py:670> took 2.481 seconds


In [19]:
ocr_result

'(i.e., physical illness, length of service, first offense) attendant to the\ncommission of the offense shall be considered. Since this is the first time\nthat respondent Sheriff Padua will be penalized for simple neglect of duty,\nthe penalty of fine in the amount of Five Thousand Pesos (25,000.00) is\nappropriate."\n\nCourt’s Rulin\n\nThe Court agrees with the OCA’s finding that Padua is guilty of\nsimple neglect of duty but increases the fine to an amount equivalent to his\nsalary for one month.\n\nSection 14, Rule 39 of the Rules of Court states that sheriffs must\nmake a report to the court every thirty days until the judgment is satisfied in\nfull:\n\nSEC. 14. Return of writ of execution. — The writ of execution shall\nbe returnable to the court issuing it immediately after the judgment has\nbeen satisfied in part or in full. If the judgment cannot be satisfied in\nfull within thirty (30) days after his receipt of the writ, the officer shall\nreport to the court and state the rea

In [15]:
import pdfplumber
pdf = pdfplumber.open("temp/temp.pdf")

In [20]:
pdf.pages[0].extract_text()

''

/Users/mv/.pyenv/versions/3.11.1/lib/python3.11/traceback.py:391: ResourceWarning: unclosed file <_io.FileIO name='temp/temp.pdf' mode='rb' closefd=True>
  yield f, (lineno, None, None, None)
